In [3]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model



In [4]:
train_dir = './chest_xray/train/'
test_dir = './chest_xray/test/'
categories = ['NORMAL', 'PNEUMONIA']


filepaths = []
labels = []

In [5]:
for category in categories:
    folder = os.path.join(train_dir, category)
    for fname in os.listdir(folder):
        filepaths.append(f"{category}/{fname}")
        labels.append(category)

In [6]:
df = pd.DataFrame({'Filename': filepaths, 'Label': labels})


In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [8]:
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [9]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col='Filename',
    y_col='Label',
    subset='training',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=(150, 150)
)

Found 4173 validated image filenames belonging to 2 classes.


In [10]:
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col='Filename',
    y_col='Label',
    subset='validation',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=(150, 150)
)

Found 1043 validated image filenames belonging to 2 classes.


In [ ]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

Class weights: {0: np.float64(7.001677852348993), 1: np.float64(0.5384516129032259)}


In [12]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


c:\Users\User\Desktop\Tutorials\Data_Science_Projects\jupyter_notebook\tensorflow_project\pneumonia_classification\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size,
    class_weight=class_weights
)

c:\Users\User\Desktop\Tutorials\Data_Science_Projects\jupyter_notebook\tensorflow_project\pneumonia_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 109s 815ms/step - accuracy: 0.4914 - auc: 0.5863 - loss: 0.6771 - val_accuracy: 0.7939 - val_auc: 0.0000e+00 - val_loss: 0.5811
Epoch 2/10
  1/130 ━━━━━━━━━━━━━━━━━━━━ 31s 248ms/step - accuracy: 0.7812 - auc: 1.0000 - loss: 0.3149

c:\Users\User\Desktop\Tutorials\Data_Science_Projects\jupyter_notebook\tensorflow_project\pneumonia_classification\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


130/130 ━━━━━━━━━━━━━━━━━━━━ 22s 170ms/step - accuracy: 0.7812 - auc: 1.0000 - loss: 0.3149 - val_accuracy: 0.7002 - val_auc: 0.0000e+00 - val_loss: 0.7834
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 94s 720ms/step - accuracy: 0.7092 - auc: 0.8111 - loss: 0.5671 - val_accuracy: 0.9277 - val_auc: 0.0000e+00 - val_loss: 0.2382
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 21s 160ms/step - accuracy: 0.8125 - auc: 0.9152 - loss: 0.4559 - val_accuracy: 0.9775 - val_auc: 0.0000e+00 - val_loss: 0.1020
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 91s 703ms/step - accuracy: 0.8167 - auc: 0.9069 - loss: 0.3983 - val_accuracy: 0.9912 - val_auc: 0.0000e+00 - val_loss: 0.0653
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 20s 155ms/step - accuracy: 0.7812 - auc: 1.0000 - loss: 0.2700 - val_accuracy: 0.9834 - val_auc: 0.0000e+00 - val_loss: 0.0912
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 90s 691ms/step - accuracy: 0.8310 - auc: 0.9127 - loss: 0.3745 - val_accuracy: 0.9180 - val_auc: 0.0000e+00 - val_loss: 0.2845
Epoch 8/1

In [15]:
test_filepaths = []
test_labels = []

In [16]:
for category in categories:
    folder = os.path.join(test_dir, category)
    for fname in os.listdir(folder):
        test_filepaths.append(f"{category}/{fname}")
        test_labels.append(category)

In [17]:
test_df = pd.DataFrame({'Filename': test_filepaths, 'Label': test_labels})

In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='Filename',
    y_col='Label',
    batch_size=32,
    shuffle=False,
    class_mode='binary',
    target_size=(150,150)
)   

Found 624 validated image filenames belonging to 2 classes.


In [20]:
preds_probs = model.predict(test_generator)
preds = (preds_probs > 0.5).astype(int).flatten()

true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# --- Метрики ---
print(classification_report(true_labels, preds, target_names=class_labels))
print(confusion_matrix(true_labels, preds))

c:\Users\User\Desktop\Tutorials\Data_Science_Projects\jupyter_notebook\tensorflow_project\pneumonia_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 359ms/step
              precision    recall  f1-score   support

      NORMAL       0.69      0.94      0.80       234
   PNEUMONIA       0.95      0.75      0.84       390

    accuracy                           0.82       624
   macro avg       0.82      0.85      0.82       624
weighted avg       0.86      0.82      0.82       624

[[220  14]
 [ 97 293]]


In [21]:
model.save("pneumonia_model.h5")
